In [1]:
import os


In [2]:
import numpy as np
from matplotlib import pyplot as plt
import nibabel as nib
from PIL import Image
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
import torchsummary
from torchvision import transforms
import time
from sklearn.metrics import classification_report

In [3]:
match_folder=np.loadtxt('/home/sumins/workspace/washout classification/base_data/condition_match_folder.txt',dtype=str)
match_folder=match_folder.tolist()

In [4]:
len(match_folder)

165

In [5]:
import ast
range_path='/home/sumins/workspace/washout classification/base_data/liverrange.txt'
range_value_path='/home/sumins/workspace/washout classification/base_data/liverrange_value.txt'

liverrange=[]
liverrange_value=[]
with open (range_path,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        liverrange.append(ast.literal_eval(line))

with open (range_value_path,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        liverrange_value.append(ast.literal_eval(line))

In [6]:
same=0
diff=0
diff_index=[]
for index, i in enumerate(liverrange_value):
    if i[0]==i[1]==i[2]==i[3]:
        same+=1
    else:
        diff+=1
        diff_index.append(index)
        

In [7]:
print("same ",same)
print("diff ", diff)

same  146
diff  19


In [8]:
diff_index

[3, 8, 10, 14, 16, 17, 22, 27, 29, 64, 77, 80, 86, 88, 113, 133, 140, 144, 151]

In [9]:
for k,i in enumerate(diff_index):
    del match_folder[i-k]
    del liverrange[i-k]
    del liverrange_value[i-k]

In [10]:
washout_slice=[]
washout_label=[]
total_slice=0
data_path='/disk1/HCC2'
for i in match_folder:    
    washout_dir = os.path.join(data_path, 'Anonymize_'+i,'washout_'+i+'.txt') 
    washout=[]
    with open(washout_dir,'r') as f:
        lines=f.readlines()
        slice=[]
        label=[]
        for line in lines:
            line=line.strip()
            washout.append(line)
        for i in washout:
            k=i.split('\t')
            total_slice+=1
            k[1]=float(k[1])
            k[1]=int(k[1])
            slice.append(int(k[0]))
            label.append(int(k[1]))
    washout_slice.append(slice) 
    washout_label.append(label)   

In [11]:
print(len(washout_slice))
print(len(washout_label))

146
146


In [12]:
zero=0
one=0
for i in washout_label:
    for j in i:
        if j==0:
            zero+=1
        elif j==1:
            one+=1
print("True: ", one)
print("False: ", zero)
print("Total slice: ",total_slice)

True:  1244
False:  404
Total slice:  1648


In [13]:
Train_washout_slice=washout_slice[:120]
Test_washout_slice=washout_slice[120:]
Train_washout_label=washout_label[:120]
Test_washout_label=washout_label[120:]

In [14]:
zero=0
one=0
for i in Train_washout_label:
    for j in i:
        if j==0:
            zero+=1
        elif j==1:
            one+=1
print("Train")
print("True: ", one)
print("False: ", zero)
print("Total slice: ",one+zero)

Train
True:  1048
False:  306
Total slice:  1354


In [15]:
zero=0
one=0
for i in Test_washout_label:
    for j in i:
        if j==0:
            zero+=1
        elif j==1:
            one+=1
print("Test")
print("True: ", one)
print("False: ", zero)
print("Total slice: ",one+zero)

Test
True:  196
False:  98
Total slice:  294


In [16]:
import ast

path1='/home/sumins/workspace/washout classification/min_slice/arterial.txt'
path2='/home/sumins/workspace/washout classification/min_slice/pre.txt'
path3='/home/sumins/workspace/washout classification/min_slice/delay.txt'
path4='/home/sumins/workspace/washout classification/min_slice/portal.txt'

arterial=[]
pre=[]
delay=[]
portal=[]

with open (path1,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        arterial.append(ast.literal_eval(line))
with open (path2,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        pre.append(ast.literal_eval(line))
with open (path3,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        delay.append(ast.literal_eval(line))
with open (path4,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        portal.append(ast.literal_eval(line))

In [17]:
for k,i in enumerate(diff_index):
    del arterial[i-k]
    del pre[i-k]
    del portal[i-k]
    del delay[i-k]

In [18]:
liverrange=[]
liverrange_value=[]

In [19]:
A_DATA=[]
R_DATA=[]
P_DATA=[]   
D_DATA=[]
all_phase=[]
LABEL=[]
data_path='/disk1/HCC2'
for i in range(len(Train_washout_label)):
    folder_name=match_folder[i]
    a_path=os.path.join(data_path, 'Anonymize_'+folder_name,'A_Anonymize_'+folder_name+'.nii') 
    a_data_all=nib.load(a_path).get_fdata()
    r_path=os.path.join(data_path, 'Anonymize_'+folder_name,'R_Anonymize_'+folder_name+'.nii') 
    r_data_all=nib.load(r_path).get_fdata()
    p_path=os.path.join(data_path, 'Anonymize_'+folder_name,'P_Anonymize_'+folder_name+'.nii') 
    p_data_all=nib.load(p_path).get_fdata()
    d_path=os.path.join(data_path, 'Anonymize_'+folder_name,'D_Anonymize_'+folder_name+'.nii') 
    d_data_all=nib.load(d_path).get_fdata()
    
    for k,j in enumerate(Train_washout_slice[i]):
        if j not in portal[i]:
            continue
        index=portal[i].index(j)
        
        r=r_data_all[:,:,pre[i][index]].astype(dtype='float32')
        a=a_data_all[:,:,arterial[i][index]].astype(dtype='float32')
        p=p_data_all[:,:,portal[i][index]].astype(dtype='float32')
        d=d_data_all[:,:,delay[i][index]].astype(dtype='float32')
        DATASET=np.stack([r,a,p,d],axis=-1)
        all_phase.append(DATASET)

        LABEL.append(Train_washout_label[i][k])
    a_data_all=0
    r_data_all=0
    p_data_all=0
    d_data_all=0
    print(i, "done")
        

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done


In [20]:
LABEL=0
r=a=p=d=0
DATASET=0
arterial=[]
pre=[]
delay=[]
portal=[]

In [21]:
print("Before: 1354")
print("After: ", len(all_phase))

Before: 1354
After:  1347


In [22]:
k=np.array(all_phase)

data=nib.Nifti1Image(k,None)

In [23]:
nib.save(data,'/home/sumins/workspace/washout classification/equal_slice/channel4_train.nii')

In [25]:
np.savetxt("/home/sumins/workspace/washout classification/equal_slice/label_train.txt",LABEL,fmt='%d')

In [19]:
A_DATAt=[]
R_DATAt=[]
P_DATAt=[]
D_DATAt=[]
LABELt=[]
all_phaset=[]
for i in range(len(Test_washout_label)):
    total_num=i+120
    folder_name=match_folder[total_num]
    a_path=os.path.join(data_path, 'Anonymize_'+folder_name,'A_Anonymize_'+folder_name+'.nii') 
    a_data_all=nib.load(a_path).get_fdata()
    r_path=os.path.join(data_path, 'Anonymize_'+folder_name,'R_Anonymize_'+folder_name+'.nii') 
    r_data_all=nib.load(r_path).get_fdata()
    p_path=os.path.join(data_path, 'Anonymize_'+folder_name,'P_Anonymize_'+folder_name+'.nii') 
    p_data_all=nib.load(p_path).get_fdata()
    d_path=os.path.join(data_path, 'Anonymize_'+folder_name,'D_Anonymize_'+folder_name+'.nii') 
    d_data_all=nib.load(d_path).get_fdata()
    
    for k,j in enumerate(Test_washout_slice[i]):
        if j not in portal[total_num]:
            continue
        index=portal[total_num].index(j)
        
        a=a_data_all[:,:,arterial[total_num][index]]
        r=r_data_all[:,:,pre[total_num][index]]
        d=d_data_all[:,:,delay[total_num][index]]
        p=p_data_all[:,:,portal[total_num][index]]
        DATASETt=np.stack([r,a,p,d],axis=-1)
        all_phaset.append(DATASETt)        

        LABELt.append(Test_washout_label[i][k])
    a_data_all=0
    r_data_all=0
    p_data_all=0
    d_data_all=0
    print(total_num, "done")
        

120 done
121 done
122 done
123 done
124 done
125 done
126 done
127 done
128 done
129 done
130 done
131 done
132 done
133 done
134 done
135 done
136 done
137 done
138 done
139 done
140 done
141 done
142 done
143 done
144 done
145 done


In [20]:
np.savetxt("/home/sumins/workspace/washout classification/equal_slice/label_test.txt",LABELt,fmt='%d')

In [21]:
print("Before: 294")
print("After: ", len(all_phaset))

Before: 294
After:  293


In [22]:
LABELt=0
r=a=p=d=0
DATASETt=0
arterial=[]
pre=[]
delay=[]
portal=[]

In [23]:
k2=np.array(all_phaset)


In [24]:
datat=nib.Nifti1Image(k2,None)

In [25]:

nib.save(datat,'/home/sumins/workspace/washout classification/equal_slice/channel4_test.nii')